scape data

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

!pip install beautifulsoup4 feedparser requests
# Iterate over each post link
import feedparser
import requests
import os.path
from bs4 import BeautifulSoup

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=15c26c0fb8bce411eaa296a93c57bce748ce24cf7a647dc08f3f28ee01018aa2
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [ ]:

# Define the RSS feed file
with open("/content/drive/MyDrive/9esa.com_rss.xml", 'r', encoding='utf-8') as file:
    sentences = file.read()
data_file = '/content/drive/MyDrive/datasets/dataset_darija_stories.txt'
# Parse the RSS feed
feed = feedparser.parse(sentences)
# Extract the post links
post_links = [entry.link for entry in feed.entries]
print(post_links)
# Print the post links
for link in post_links:
    post_response = requests.get(link)
    if post_response.status_code != 200:
        print("Error accessing post:", link)
        continue

    # Create BeautifulSoup object with the post's HTML content
    post_soup = BeautifulSoup(post_response.content, 'html.parser')

    # Find all ".Noto p" elements and extract their text
    paragraphs = post_soup.select('.Noto p')
    text = '\n'.join([p.get_text(strip=True) for p in paragraphs])

    # Save the extracted text to the output file
    with open(data_file, 'a', encoding='utf-8') as file:
      file.write(text + '\n\n')

['https://www.9esa.com/2023/07/la-tojbirni-3ala-7obik-20.html', 'https://www.9esa.com/2023/07/3ers-dib-28.html', 'https://www.9esa.com/2023/07/3asifa-hawja2-28.html', 'https://www.9esa.com/2023/07/ghirk-ma-y7lali-28.html', 'https://www.9esa.com/2023/07/la-tojbirni-3ala-7obik-19.html', 'https://www.9esa.com/2023/07/3ers-dib-27.html', 'https://www.9esa.com/2023/07/3asifa-hawja2-27.html', 'https://www.9esa.com/2023/07/ghirk-ma-y7lali-27.html', 'https://www.9esa.com/2023/07/la-tojbirni-3ala-7obik-18.html', 'https://www.9esa.com/2023/07/3ers-dib-26.html', 'https://www.9esa.com/2023/07/3asifa-hawja2-26.html', 'https://www.9esa.com/2023/07/ghirk-ma-y7lali-26.html', 'https://www.9esa.com/2023/07/la-tojbirni-3ala-7obik-17.html', 'https://www.9esa.com/2023/07/3ers-dib-25.html', 'https://www.9esa.com/2023/07/3asifa-hawja2-25.html', 'https://www.9esa.com/2023/07/ghirk-ma-y7lali-25.html', 'https://www.9esa.com/2023/07/la-tojbirni-3ala-7obik-16.html', 'https://www.9esa.com/2023/07/3ers-dib-24.html',

> **Finetuning gpt2 on darija sentence's with tests included**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install required packages
!pip install torch torchvision transformers
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, AdamW

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.2 MB/s eta 0:00:00


In [ ]:
# !jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000
# Step 1: Load and preprocess the dataset
data_file = '/content/drive/MyDrive/datasets/dataset_darija_stories.txt'

with open(data_file, 'r', encoding='utf-8') as file:
    sentences = file.read().splitlines()
    sentences = [el.replace('"', '') for el in sentences]
    print(cleaned_sentences[0:10])

['سأخبرك سرا', 'إنني أخون كبريائي في الخفاء', 'وأتبع أخبارك ليطمئن قلبي عليك', 'اشتقت لك بصمت  فهل يصلك ضجيج حنيني', 'رسالتها كانت كالبلسم على جروحوو و رووحو كان في قمة احتيااجو ليها تمنى يعيش معاها ولو غير لحظة هانيين و تكمل فرحتهم لكن الطرووف دائما ضده اكيد حتى هو ندم على تهوره و زوااجو يمكن مكانش غايظلم معاه زوج ديال النااس معاه وحدة تيموت عليها و هي مزالها متضااربة مع احساسها و وحدة تتبغييه و هو تايحس فقط بالواجب تجاهها', 'اما مريم وصلت لمرحة اصبحت تتقول حزنت بما يكفي وبكيت على الاطلال زمنا طويلا و جا الوقت فين تفتح الباب لقلبها و تعطي فرصة لدااك ولد النااس يدخل ليه  لكن صدمة الاخيرة ديال فائزة خلاتها تفكر فحااجة وحدة هي الفرااق غاافلة على ان هااد الرجل مااشي وقت لي قالت لو اجي يجي تيجري و وقت لي بغات تقوول ليه سالينا غايمشي بكل سهوولة الانساان عندوو طااقة التحمل محدودة و خصووصا الرجال تتجي وااحد اللحظة الغضب تيعمي على العين الانساان متيبقاش عارف شنو تيدير لكن هاد المرة حتى طلب الطلاق ندمات عليه الاياام لي فااتو متيزورهاش كيف عادتوو مرو كانو كانهم دهر هل هو الشووق الحنين اكيد الندم

In [ ]:
# Set up GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Step 2: Tokenize the sentences
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenized_sentences = [tokenizer.encode(sentence) for sentence in sentences]

# Step 3: Define a custom PyTorch Dataset
class DarijaDataset(Dataset):
    def __init__(self, tokenized_sentences):
        self.tokenized_sentences = tokenized_sentences

    def __len__(self):
        return len(self.tokenized_sentences)

    def __getitem__(self, idx):
        return torch.tensor(self.tokenized_sentences[idx])

# Step 4: Prepare the GPT-2 model
config = GPT2Config.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2', config=config)
model.to(device)

# Step 5: Set up the fine-tuning pipeline
dataset = DarijaDataset(tokenized_sentences)
batch_size = 4
num_epochs = 3

# Define custom collate function to pad sequences dynamically
def collate_fn(batch):
    padded_batch = pad_sequence(batch, batch_first=True)
    return padded_batch

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

optimizer = AdamW(model.parameters(), lr=1e-5)

# Step 6: Fine-tune the GPT-2 model
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch in dataloader:
        batch = batch.to(device)
        outputs = model.forward(input_ids=batch[:, :-1], labels=batch[:, 1:])
        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch + 1} - Average Loss: {avg_loss:.4f}")

# Step 7: Save the fine-tuned model
output_dir = '/content/drive/MyDrive/dataset/fine_tuned_model'
model.save_pretrained(output_dir)



NameError: ignored

__*now let test our model*__

In [ ]:
# Step 8: Inference and generation using the fine-tuned model
Load the fine-tuned model for future use
fine_tuned_model = GPT2LMHeadModel.from_pretrained(output_dir)

# Generate text using the fine-tuned model
inputs = tokenizer.encode("Your input prompt", return_tensors="pt").to(device)
generated_text = fine_tuned_model.generate(inputs, max_length=100)